Clone the repository


In [ ]:
! git clone https://github.com/KwaiVGI/LivePortrait.git

fatal: destination path 'LivePortrait' already exists and is not an empty directory.


In [ ]:
!cd LivePortrait && pip install -r requirements.txt

  Using cached gradio-5.1.0-py3-none-any.whl.metadata (15 kB)
  Using cached gradio_client-1.4.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached websockets-12.0-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
Using cached gradio-5.1.0-py3-none-any.whl (42.3 MB)
Using cached gradio_client-1.4.0-py3-none-any.whl (319 kB)
Using cached websockets-12.0-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (131 kB)
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.13.3
    Uninstalling gradio_client-1.13.3:
      Successfully uninstalled gradio_client-1.13.3
  Attempting uninstall: gradio
    Found existing installation: gradio 5.48.0
    Uninstalling gradio-5.48.0:
      Suc

In [ ]:
!pip install -U "huggingface_hub[cli]"


In [ ]:
# !huggingface-cli download KwaiVGI/LivePortrait --local-dir pretrained_weights --exclude "*.git*" "README.md" "docs"

In [ ]:
!huggingface-cli download KwaiVGI/LivePortrait --local-dir /content/LivePortrait/pretrained_weights --exclude "*.git*" "README.md" "docs"

⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
Fetching 20 files: 100% 20/20 [00:00<00:00, 6838.35it/s]
/content/LivePortrait/pretrained_weights


In [ ]:
!pip install -U gradio

  Using cached gradio-5.48.0-py3-none-any.whl.metadata (16 kB)
  Using cached gradio_client-1.13.3-py3-none-any.whl.metadata (7.1 kB)
  Using cached websockets-15.0.1-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached gradio-5.48.0-py3-none-any.whl (63.5 MB)
Using cached gradio_client-1.13.3-py3-none-any.whl (325 kB)
Using cached websockets-15.0.1-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (182 kB)
  Attempting uninstall: websockets
    Found existing installation: websockets 12.0
    Uninstalling websockets-12.0:
      Successfully uninstalled websockets-12.0
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.4.0
    Uninstalling gradio_client-1.4.0:
      Successfully uninstalled gradio_client-1.4.0
  Attempting uninstall: gradio
    Found existing installation: gradio 5.1.0
    Uninstalling gradio-5.1.0:
      Succes

In [ ]:
!pip install gradio gradio-client

##Gradio Code

In [ ]:
"""Importing necessary libraries"""
import sys
sys.path.append('/content/LivePortrait')
import os
import tyro
import tempfile
import subprocess
import gradio as gr
import os.path as osp
from os.path import basename

from src.utils.helper import load_description
from src.gradio_pipeline import GradioPipeline
from src.config.crop_config import CropConfig
from src.config.argument_config import ArgumentConfig
from src.config.inference_config import InferenceConfig
from src.live_portrait_pipeline import LivePortraitPipeline

project_root = '/content/LivePortrait'
src_dir = os.path.join(project_root, 'src')
sys.path.append(src_dir)

def partial_fields(target_class, kwargs):
    return target_class(**{k: v for k, v in kwargs.items() if hasattr(target_class, k)})

def fast_check_ffmpeg():
    try:
        subprocess.run(["ffmpeg", "-version"], capture_output=True, check=True)
        return True
    except:
        return False

def fast_check_args(args: ArgumentConfig):
    if not os.path.exists(args.source):
        raise FileNotFoundError(f"Source info not found: {args.source}")
    if not os.path.exists(args.driving):
        raise FileNotFoundError(f"Driving info not found: {args.driving}")

def prefix(filename):
    pos = filename.rfind(".")
    if pos == -1:
        return filename
    return filename[:pos]

def basename(filename):
    return prefix(os.path.basename(filename))

def run_live_portrait_pipeline(source_image_path, driving_video_path):
    print(f"Source image path: {source_image_path}")
    print(f"Driving video path: {driving_video_path}")

    if not os.path.exists(source_image_path):
        return f"Source image not found: {source_image_path}"
    if not os.path.exists(driving_video_path):
        return f"Driving video not found: {driving_video_path}"

    args = ArgumentConfig(source=source_image_path, driving=driving_video_path)
    print(f"Arguments: {args}")

    if not hasattr(args, 'output_dir'):
        return "Output directory not specified in argument configuration."

    ffmpeg_dir = os.path.join(os.getcwd(), "ffmpeg")
    if os.path.exists(ffmpeg_dir):
        os.environ["PATH"] += (os.pathsep + ffmpeg_dir)

    if not fast_check_ffmpeg():
        return "FFmpeg is not installed. Please install FFmpeg."

    try:
        fast_check_args(args)
    except FileNotFoundError as e:
        return str(e)

    inference_cfg = partial_fields(InferenceConfig, args.__dict__)
    crop_cfg = partial_fields(CropConfig, args.__dict__)

    live_portrait_pipeline = LivePortraitPipeline(
        inference_cfg=inference_cfg,
        crop_cfg=crop_cfg
    )

    live_portrait_pipeline.execute(args)
    output_file_path = os.path.join(args.output_dir, f'{basename(args.source)}--{basename(args.driving)}.mp4')
    print(f"Output video path: {output_file_path}")

    return output_file_path

tyro.extras.set_accent_color("bright_cyan")

args = ArgumentConfig(source='')
ffmpeg_dir = os.path.join(os.getcwd(), "ffmpeg")

if osp.exists(ffmpeg_dir):
    os.environ["PATH"] += (os.pathsep + ffmpeg_dir)

if not fast_check_ffmpeg():
    raise ImportError(
        "FFmpeg is not installed. Please install FFmpeg (including ffmpeg and ffprobe) before running this script. https://ffmpeg.org/download.html"
    )

inference_cfg = partial_fields(InferenceConfig, args.__dict__)
crop_cfg = partial_fields(CropConfig, args.__dict__)

gradio_pipeline = GradioPipeline(
    inference_cfg=inference_cfg,
    crop_cfg=crop_cfg,
    args=args
)

def gpu_wrapped_execute_image_retargeting(*args, **kwargs):
    return gradio_pipeline.execute_image_retargeting(*args, **kwargs)

def reset_sliders(*args, **kwargs):
    return 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.5, True, True

# Assets paths
title_md = "/content/LivePortrait/assets/gradio/gradio_title_1.md"
example_portrait_dir = "/content/LivePortrait/assets/examples/source"
example_video_dir = "/content/LivePortrait/assets/examples/driving"

# Custom CSS for complete aesthetic transformation
custom_css = """
@import url('https://fonts.googleapis.com/css2?family=Space+Grotesk:wght@400;500;600;700&display=swap');

* {
    font-family: 'Space Grotesk', sans-serif !important;
}

.gradio-container {
    background: linear-gradient(135deg, #0a0e27 0%, #1a1d3a 50%, #0f1129 100%) !important;
    color: #e0e7ff !important;
}

.contain {
    background: rgba(15, 17, 41, 0.6) !important;
    backdrop-filter: blur(10px) !important;
}

/* Glowing neon accent colors */
:root {
    --primary-glow: #00f5ff;
    --secondary-glow: #ff00ff;
    --tertiary-glow: #7000ff;
}

/* Headers and text */
h1, h2, h3, h4, h5 {
    background: linear-gradient(90deg, var(--primary-glow), var(--secondary-glow), var(--tertiary-glow));
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    background-clip: text;
    font-weight: 700 !important;
    letter-spacing: 1px;
    text-shadow: 0 0 20px rgba(0, 245, 255, 0.3);
}

/* Buttons */
.gr-button {
    background: linear-gradient(135deg, #00f5ff 0%, #7000ff 100%) !important;
    border: none !important;
    color: #ffffff !important;
    font-weight: 600 !important;
    text-transform: uppercase;
    letter-spacing: 1.5px;
    box-shadow: 0 0 20px rgba(0, 245, 255, 0.4), 0 0 40px rgba(112, 0, 255, 0.2) !important;
    transition: all 0.3s ease !important;
    border-radius: 12px !important;
    padding: 12px 24px !important;
}

.gr-button:hover {
    box-shadow: 0 0 30px rgba(0, 245, 255, 0.6), 0 0 60px rgba(255, 0, 255, 0.4) !important;
    transform: translateY(-2px) !important;
}

.gr-button-primary {
    background: linear-gradient(135deg, #ff00ff 0%, #00f5ff 100%) !important;
}

/* Input fields */
.gr-input, .gr-text-input, .gr-number-input {
    background: rgba(26, 29, 58, 0.8) !important;
    border: 2px solid rgba(0, 245, 255, 0.3) !important;
    color: #e0e7ff !important;
    border-radius: 10px !important;
    transition: all 0.3s ease !important;
}

.gr-input:focus, .gr-text-input:focus, .gr-number-input:focus {
    border-color: var(--primary-glow) !important;
    box-shadow: 0 0 15px rgba(0, 245, 255, 0.3) !important;
}

/* Sliders */
.gr-slider input[type="range"] {
    background: linear-gradient(90deg, rgba(0, 245, 255, 0.2), rgba(255, 0, 255, 0.2)) !important;
}

.gr-slider input[type="range"]::-webkit-slider-thumb {
    background: linear-gradient(135deg, #00f5ff, #ff00ff) !important;
    box-shadow: 0 0 10px rgba(0, 245, 255, 0.6) !important;
    border: none !important;
    width: 20px !important;
    height: 20px !important;
}

/* Accordions */
.gr-accordion {
    background: rgba(26, 29, 58, 0.5) !important;
    border: 1px solid rgba(0, 245, 255, 0.2) !important;
    border-radius: 15px !important;
    backdrop-filter: blur(5px) !important;
}

.gr-accordion-header {
    background: linear-gradient(135deg, rgba(0, 245, 255, 0.1), rgba(112, 0, 255, 0.1)) !important;
    color: var(--primary-glow) !important;
    font-weight: 600 !important;
    border-radius: 15px 15px 0 0 !important;
}

/* Tabs */
.gr-tabs {
    background: rgba(15, 17, 41, 0.8) !important;
    border-radius: 15px !important;
}

.gr-tab {
    background: rgba(26, 29, 58, 0.4) !important;
    color: #8892b8 !important;
    border: 1px solid rgba(0, 245, 255, 0.1) !important;
    border-radius: 10px 10px 0 0 !important;
    transition: all 0.3s ease !important;
}

.gr-tab.selected {
    background: linear-gradient(135deg, rgba(0, 245, 255, 0.2), rgba(112, 0, 255, 0.2)) !important;
    color: var(--primary-glow) !important;
    border-bottom: 3px solid var(--primary-glow) !important;
    box-shadow: 0 -5px 20px rgba(0, 245, 255, 0.2) !important;
}

/* Checkboxes and Radio buttons */
.gr-checkbox, .gr-radio {
    background: rgba(26, 29, 58, 0.6) !important;
    border-radius: 10px !important;
    padding: 10px !important;
}

input[type="checkbox"]:checked {
    background: linear-gradient(135deg, #00f5ff, #7000ff) !important;
    border: none !important;
}

input[type="radio"]:checked {
    background: var(--secondary-glow) !important;
    box-shadow: 0 0 10px rgba(255, 0, 255, 0.5) !important;
}

/* Video and Image containers */
.gr-video, .gr-image {
    border: 2px solid rgba(0, 245, 255, 0.3) !important;
    border-radius: 15px !important;
    box-shadow: 0 0 20px rgba(0, 245, 255, 0.1) !important;
    background: rgba(10, 14, 39, 0.8) !important;
}

/* Rows and columns */
.gr-row {
    gap: 20px !important;
}

.gr-column {
    background: rgba(26, 29, 58, 0.3) !important;
    border-radius: 15px !important;
    padding: 15px !important;
}

/* Labels */
label {
    color: #b4c6fc !important;
    font-weight: 500 !important;
    text-transform: uppercase;
    font-size: 0.85rem !important;
    letter-spacing: 0.5px;
}

/* Markdown content */
.gr-markdown {
    color: #e0e7ff !important;
    background: rgba(26, 29, 58, 0.4) !important;
    border-left: 3px solid var(--primary-glow) !important;
    padding: 15px !important;
    border-radius: 10px !important;
}

/* Examples */
.gr-examples {
    background: rgba(15, 17, 41, 0.6) !important;
    border-radius: 12px !important;
    padding: 10px !important;
}

/* Scrollbar styling */
::-webkit-scrollbar {
    width: 10px;
    background: rgba(10, 14, 39, 0.8);
}

::-webkit-scrollbar-thumb {
    background: linear-gradient(180deg, #00f5ff, #7000ff);
    border-radius: 5px;
}

/* Glow effects on hover */
.gr-button:hover, .gr-accordion:hover, .gr-tab:hover {
    filter: brightness(1.2);
}

/* Animation */
@keyframes pulse {
    0%, 100% { opacity: 1; }
    50% { opacity: 0.7; }
}

.gr-button-primary {
    animation: pulse 2s ease-in-out infinite;
}
"""

# Create the interface with new theme
with gr.Blocks(css=custom_css, theme=gr.themes.Base()) as demo:
    gr.Markdown("# 🎭 LIVEPORTRAIT STUDIO", elem_classes="main-title")
    gr.Markdown(load_description("/content/LivePortrait/assets/gradio/gradio_description_upload.md"))

    with gr.Row():
        with gr.Column(scale=1):
            with gr.Tabs():
                with gr.TabItem("🖼️ SOURCE IMAGE") as tab_image:
                    with gr.Accordion(open=True, label="📸 Upload Source"):
                        source_image_input = gr.Image(type="filepath")
                        gr.Examples(
                            examples=[
                                [osp.join(example_portrait_dir, "s0.jpg")],
                                [osp.join(example_portrait_dir, "s10.jpg")],
                                [osp.join(example_portrait_dir, "s7.jpg")],
                                [osp.join(example_portrait_dir, "s22.jpg")],
                                [osp.join(example_portrait_dir, "s23.jpg")],
                            ],
                            inputs=[source_image_input],
                            cache_examples=False,
                        )

        with gr.Column(scale=1):
            with gr.Tabs():
                with gr.TabItem("🎬 DRIVING VIDEO") as v_tab_video:
                    with gr.Accordion(open=True, label="🎥 Upload Driver"):
                        driving_video_input = gr.Video()
                        gr.Examples(
                            examples=[
                                [osp.join(example_video_dir, "d0.mp4")],
                                [osp.join(example_video_dir, "d18.mp4")],
                                [osp.join(example_video_dir, "d19.mp4")],
                                [osp.join(example_video_dir, "d6.mp4")],
                                [osp.join(example_video_dir, "d20.mp4")],
                            ],
                            inputs=[driving_video_input],
                            cache_examples=False,
                        )

                v_tab_selection = gr.Textbox(visible=False)
                v_tab_video.select(lambda: "Video", None, v_tab_selection)

    with gr.Row():
        with gr.Accordion(open=False, label="⚙️ ADVANCED CONFIGURATION"):
            with gr.Row():
                flag_normalize_lip = gr.Checkbox(value=False, label="Normalize Lip")
                flag_relative_input = gr.Checkbox(value=True, label="Relative Motion")
                flag_remap_input = gr.Checkbox(value=True, label="Paste-Back")
                flag_stitching_input = gr.Checkbox(value=True, label="Stitching")
            with gr.Row():
                animation_region = gr.Radio(["exp", "pose", "lip", "eyes", "all"], value="all", label="Animation Region")
                driving_option_input = gr.Radio(['expression-friendly', 'pose-friendly'], value="expression-friendly", label="Driving Mode")
            with gr.Row():
                driving_multiplier = gr.Number(value=1.0, label="Driving Multiplier", minimum=0.0, maximum=2.0, step=0.02)
                driving_smooth_observation_variance = gr.Number(value=3e-7, label="Motion Smoothing", minimum=1e-11, maximum=1e-2, step=1e-8)

    gr.Markdown(load_description("/content/LivePortrait/assets/gradio/gradio_description_animate_clear.md"))

    with gr.Row():
        process_button_animation = gr.Button("🚀 GENERATE ANIMATION", variant="primary", size="lg")

    with gr.Row():
        with gr.Column():
            output_file_path = gr.Video(autoplay=False, label="✨ Animated Result", height=360)

    with gr.Row():
        process_button_reset = gr.ClearButton([source_image_input, driving_video_input, output_file_path], value="🔄 RESET ALL")

    process_button_animation.click(
        fn=run_live_portrait_pipeline,
        inputs=[source_image_input, driving_video_input],
        outputs=output_file_path
    )

    # Retargeting section with new styling
    gr.Markdown("## 🎯 FACIAL RETARGETING STUDIO")

    with gr.Row(visible=True):
        flag_do_crop_input_retargeting_image = gr.Checkbox(value=True, label="Enable Crop")
        flag_stitching_retargeting_input = gr.Checkbox(value=True, label="Enable Stitching")
        retargeting_source_scale = gr.Number(minimum=1.8, maximum=3.2, value=2.5, step=0.05, label="Crop Scale")
        eye_retargeting_slider = gr.Slider(minimum=0, maximum=0.8, step=0.01, label="👁️ Eyes Open Ratio")
        lip_retargeting_slider = gr.Slider(minimum=0, maximum=0.8, step=0.01, label="👄 Lip Open Ratio")

    with gr.Row(visible=True):
        with gr.Column():
            with gr.Accordion(open=False, label="🎭 HEAD MOVEMENT CONTROLS"):
                head_pitch_slider = gr.Slider(minimum=-15.0, maximum=15.0, value=0, step=1, label="⬆️ Pitch")
                head_yaw_slider = gr.Slider(minimum=-25, maximum=25, value=0, step=1, label="↔️ Yaw")
                head_roll_slider = gr.Slider(minimum=-15.0, maximum=15.0, value=0, step=1, label="🔄 Roll")
                mov_x = gr.Slider(minimum=-0.19, maximum=0.19, value=0.0, step=0.01, label="↔️ X-Axis")
                mov_y = gr.Slider(minimum=-0.19, maximum=0.19, value=0.0, step=0.01, label="↕️ Y-Axis")
                mov_z = gr.Slider(minimum=0.9, maximum=1.2, value=1.0, step=0.01, label="🔍 Z-Axis")

        with gr.Column():
            with gr.Accordion(open=False, label="😊 EXPRESSION CONTROLS"):
                lip_variation_zero = gr.Slider(minimum=-0.09, maximum=0.09, value=0, step=0.01, label="💋 Pouting")
                lip_variation_one = gr.Slider(minimum=-20.0, maximum=15.0, value=0, step=0.01, label="👄 Pursing")
                lip_variation_two = gr.Slider(minimum=0.0, maximum=15.0, value=0, step=0.01, label="😁 Grin")
                lip_variation_three = gr.Slider(minimum=-90.0, maximum=120.0, value=0, step=1.0, label="🗣️ Mouth Open")
                smile = gr.Slider(minimum=-0.3, maximum=1.3, value=0, step=0.01, label="😊 Smile")
                wink = gr.Slider(minimum=0, maximum=39, value=0, step=0.01, label="😉 Wink")
                eyebrow = gr.Slider(minimum=-30, maximum=30, value=0, step=0.01, label="🤨 Eyebrow")
                eyeball_direction_x = gr.Slider(minimum=-30.0, maximum=30.0, value=0, step=0.01, label="👀 Gaze H")
                eyeball_direction_y = gr.Slider(minimum=-63.0, maximum=63.0, value=0, step=0.01, label="👁️ Gaze V")

    with gr.Row(visible=True):
        reset_button = gr.Button("🔄 RESET SLIDERS")
        reset_button.click(
            fn=reset_sliders,
            inputs=None,
            outputs=[
                head_pitch_slider, head_yaw_slider, head_roll_slider, mov_x, mov_y, mov_z,
                lip_variation_zero, lip_variation_one, lip_variation_two, lip_variation_three,
                smile, wink, eyebrow, eyeball_direction_x, eyeball_direction_y,
                retargeting_source_scale, flag_stitching_retargeting_input, flag_do_crop_input_retargeting_image
            ]
        )

    with gr.Row(visible=True):
        with gr.Column():
            with gr.Accordion(open=True, label="📥 INPUT"):
                retargeting_input_image = gr.Image(type="numpy")
        with gr.Column():
            with gr.Accordion(open=True, label="🎨 RETARGETED"):
                retargeting_output_image = gr.Image(type="numpy")
        with gr.Column():
            with gr.Accordion(open=True, label="✅ FINAL RESULT"):
                retargeting_output_image_paste_back = gr.Image(type="numpy")

    with gr.Row(visible=True):
        process_button_reset_retargeting = gr.ClearButton(
            [retargeting_input_image, retargeting_output_image, retargeting_output_image_paste_back],
            value="🗑️ CLEAR RETARGETING"
        )

    retargeting_input_image.change(
        fn=gradio_pipeline.init_retargeting_image,
        inputs=[retargeting_source_scale, eye_retargeting_slider, lip_retargeting_slider, retargeting_input_image],
        outputs=[eye_retargeting_slider, lip_retargeting_slider]
    )

    sliders = [eye_retargeting_slider, lip_retargeting_slider, head_pitch_slider, head_yaw_slider,
               head_roll_slider, mov_x, mov_y, mov_z, lip_variation_zero, lip_variation_one,
               lip_variation_two, lip_variation_three, smile, wink, eyebrow, eyeball_direction_x,
               eyeball_direction_y]

    for slider in sliders:
        slider.change(
            fn=gpu_wrapped_execute_image_retargeting,
            inputs=[
                eye_retargeting_slider, lip_retargeting_slider, head_pitch_slider, head_yaw_slider,
                head_roll_slider, mov_x, mov_y, mov_z, lip_variation_zero, lip_variation_one,
                lip_variation_two, lip_variation_three, smile, wink, eyebrow, eyeball_direction_x,
                eyeball_direction_y, retargeting_input_image, retargeting_source_scale,
                flag_stitching_retargeting_input, flag_do_crop_input_retargeting_image
            ],
            outputs=[retargeting_output_image, retargeting_output_image_paste_back],
        )

if __name__ == "__main__":
    demo.launch(debug=True)

[03:52:54] Load appearance_feature_extractor from                                       ]8;id=652761;file:///content/LivePortrait/src/live_portrait_wrapper.py\live_portrait_wrapper.py]8;;\:]8;id=282996;file:///content/LivePortrait/src/live_portrait_wrapper.py#46\46]8;;\
           /content/LivePortrait/pretrained_weights/liveportrait/base_models/appearance                            
           _feature_extractor.pth done.                                                                            

[03:52:55] Load motion_extractor from                                                   ]8;id=817864;file:///content/LivePortrait/src/live_portrait_wrapper.py\live_portrait_wrapper.py]8;;\:]8;id=966371;file:///content/LivePortrait/src/live_portrait_wrapper.py#49\49]8;;\
           /content/LivePortrait/pretrained_weights/liveportrait/base_models/motion_ext                            
           ractor.pth done.                                                                                        

[03:52:56] Load warping_module from                                                     ]8;id=510127;file:///content/LivePortrait/src/live_portrait_wrapper.py\live_portrait_wrapper.py]8;;\:]8;id=699105;file:///content/LivePortrait/src/live_portrait_wrapper.py#52\52]8;;\
           /content/LivePortrait/pretrained_weights/liveportrait/base_models/warping_mo                            
           dule.pth done.                                                                                          

[03:52:57] Load spade_generator from                                                    ]8;id=763896;file:///content/LivePortrait/src/live_portrait_wrapper.py\live_portrait_wrapper.py]8;;\:]8;id=156654;file:///content/LivePortrait/src/live_portrait_wrapper.py#55\55]8;;\
           /content/LivePortrait/pretrained_weights/liveportrait/base_models/spade_gene                            
           rator.pth done.                                                                                         

           Load stitching_retargeting_module from                                       ]8;id=63302;file:///content/LivePortrait/src/live_portrait_wrapper.py\live_portrait_wrapper.py]8;;\:]8;id=957406;file:///content/LivePortrait/src/live_portrait_wrapper.py#59\59]8;;\
           /content/LivePortrait/pretrained_weights/liveportrait/retargeting_models/sti                            
           tching_retargeting_module.pth done.                                                                     

...

[04:59:25] Dump motion template to                                                    ]8;id=802464;file:///content/LivePortrait/src/live_portrait_pipeline.py\live_portrait_pipeline.py]8;;\:]8;id=912958;file:///content/LivePortrait/src/live_portrait_pipeline.py#172\172]8;;\
           /tmp/gradio/ba395ca06da11b5773ae7974d57141107d2fa35fc7cc364b13728374167078                              
           2d/d6.pkl                                                                                               

           Prepared pasteback mask done.                                              ]8;id=418988;file:///content/LivePortrait/src/live_portrait_pipeline.py\live_portrait_pipeline.py]8;;\:]8;id=610228;file:///content/LivePortrait/src/live_portrait_pipeline.py#183\183]8;;\

[04:59:28] The animated video consists of 1008 frames.                                ]8;id=638830;file:///content/LivePortrait/src/live_portrait_pipeline.py\live_portrait_pipeline.py]8;;\:]8;id=639106;file:///content/LivePortrait/src/live_portrait_pipeline.py#270\270]8;;\

Output()

Output()